## Leave-word-out

This script gets a relevant score for each word, removing the word from test dataset and obtaining the performance metrics for the classification of those customized tweets.

The score is calculated as:
```
custom_dataset_f1_score - original_dataset_f1_score
```
If the result is positive: removing the word benefits the results, so the word **IS NOT HELPING** in the classification.
If the result is negative: removing the word damages the results, so the word **IS HELPING** in the classification.

They are being considerated only words that previously were in the top 500 words with greatest absolute attention.

In [ ]:
import pandas as pd
from pathlib import Path
import time
import datetime

BATCH_SIZE = 16

DATASET_FOLDER = 'DS3'
CHECKPOINT = 'bert-base-multilingual-uncased'
FOLDER = './outputs/DS3/bert-base-multilingual-uncased'

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


# Import words
words = pd.read_csv(f'{FOLDER}/words/top_500_words.csv', sep=';').word.values

### 1) Set-up model

In [ ]:
import torch
from transformers import BertForSequenceClassification, AutoTokenizer, DataCollatorWithPadding

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

saved_model = f'{FOLDER}/model/pytorch_model.bin'
saved_config = f'{FOLDER}/model/config.json'

model = BertForSequenceClassification.from_pretrained(
    saved_model,
    config=saved_config
)
model.to(device)
print(f'Model is designed for {model.num_labels} labels.')
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def tokenize_function(instance):
    return tokenizer(instance['tweet'], truncation=True)

device

### 2) Load dataset

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load/Calculate performance metrics for dataset
test_results = pd.read_csv(f'{FOLDER}/test_results_complete.csv', sep=';')

original_test_scores = {
    'accuracy': accuracy_score(test_results.expected.values, test_results.got.values),
    'precision': precision_score(test_results.expected.values, test_results.got.values, average='weighted'),
    'recall': recall_score(test_results.expected.values, test_results.got.values, average='weighted'),
    'f1': f1_score(test_results.expected.values, test_results.got.values, average='weighted'),
}
original_test_scores

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    'csv',
    data_files={
        f'test': f'./datasets/{DATASET_FOLDER}/test_dataset.csv',
    }
)
dataset

### 3) Perform leave-word-out

In [ ]:
from torch.utils.data import DataLoader
from datasets import load_metric
import re

words_lwo = {}

metrics = {
    'accuracy': load_metric('accuracy'),
    'precision': load_metric('precision'),
    'recall': load_metric('recall'),
    'f1': load_metric('f1'),
}

model.eval()

t0 = time.time() 

for step, word in enumerate(words):
    
    # Remove word from tweet and tokenize it
    tokenized_data = dataset.map(
        (
            lambda instance: tokenizer(
                [ re.sub(fr'\b{word}\b', '', tweet) for tweet in instance['tweet'] ],
                truncation=True
            )
        ),
        batched=True
    )
    
    tokenized_data = tokenized_data.remove_columns(
        ['tweet']
    )
    tokenized_data = tokenized_data.rename_column('label', 'labels')
    tokenized_data.set_format('torch')
    
    # Create DataLoader
    test_dataloader = DataLoader(
        tokenized_data['test'], batch_size=BATCH_SIZE, collate_fn=data_collator
    )
    
    eval_metrics = metrics

    # Predict for custom test dataset
    for i, batch in enumerate(test_dataloader):
    
        batch = {k: v.to(device) for k, v in batch.items()}    

        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        
        for _, metric in eval_metrics.items():
            metric.add_batch(predictions=predictions, references=batch['labels'])

    words_lwo[word] = {
        'word': word,
        'accuracy': eval_metrics['accuracy'].compute()['accuracy'] - original_test_scores['accuracy'],
        'precision': eval_metrics['precision'].compute(average='weighted')['precision'] - original_test_scores['precision'],
        'recall': eval_metrics['recall'].compute(average='weighted')['recall'] - original_test_scores['recall'],
        'f1': eval_metrics['f1'].compute(average='weighted')['f1'] - original_test_scores['f1'],
    }
    
    if step % 20 == 0:
        # Calculate elapsed time in minutes.
        elapsed = format_time(time.time() - t0)
        # Report progress.
        print('Word {:>3,}  of  {:>3,}. "{:}", took: {:}'.format(step, len(words), word, elapsed))

### 4) Save words an leave-word-out scores

In [ ]:
lwo_df = pd.DataFrame(words_lwo).T
lwo_df.to_csv(f'{FOLDER}/lwo.csv', index=None)
lwo_df

In [ ]:
# lwo_df.sort_values('f1', ascending=True)
lwo_df[lwo_df.f1 < 0]